# Recurrent neural networks

[Recurrent neural networks (RNNs)](https://en.wikipedia.org/wiki/Recurrent_neural_network) are a class of neural networks where connections between nodes (units/neurons) can create a directed cycle so that output from some nodes can affect subsequent input to the same node. RNNs are able to maintain an internal state that captures information about the sequence seen so far and they are able to use this internal state to make predictions about the next element in the sequence. This internal state is updated based on the input at each time step, and is used to make predictions about the next element in the sequence. RNNs are commonly used for processing _sequential data_ such as text in natural language processing (NLP), speech in speech recognition, and time series in time series (e.g. stock price, weather) forecasting.

Watch the 15-minute video below for a visual explanation of recurrent neural networks.

```{admonition} Video
<iframe width="700" height="394" src="https://www.youtube.com/embed/AsNTP8Kwu80?start=45&end=953" title="YouTube video player" frameborder="0" allow="accelerometer; autoplay; clipboard-write; encrypted-media; gyroscope; picture-in-picture" allowfullscreen></iframe>

[Explaining main ideas behind recurrent neural networks by StatQuest](https://www.youtube.com/embed/AsNTP8Kwu80?start=45&end=953), embedded according to [YouTube's Terms of Service](https://www.youtube.com/static?gl=CA&template=terms).
```
Remove or comment off the following installation if you have installed PyTorch and TorchVision already.

In [ ]:
!pip3 install -q torch torchvision

## Why recurrent neural networks?

In the previous section, we saw how to use convolutional neural networks (CNNs) to process images. CNNs are a type of neural network that is particularly well suited to processing data that has a grid-like structure, such as images. 

In this section, we will see how to use recurrent neural networks (RNNs) to process sequential data. RNNs are a type of neural network that is particularly well suited to processing data that has a temporal structure, such as text. In particular, they are able to maintain an internal state that captures information about the sequence seen so far. They can process sequences of variable length, unlike CNNs which can only process fixed-size inputs.

There are two key ideas behind RNNs:

- **Recurrent connections**. RNNs have recurrent connections that allow information to flow through the network in a directed cycle. This allows the network to use its internal state to make predictions about the next element in the sequence.
- **Weight sharing**. RNNs share weights across time steps. This means that the same weights are used to process the first element of the sequence, the second element, the third element, and so on. This allows the network to efficiently represent patterns that are consistent across time steps.

## Unfolding a recurrent neural network

{numref}`rnn-unfold` shows a recurrent neural network in the compressed form on the left and the unfolded form for three time steps on the right. At each time step, the network takes an input and produces an output and a hidden state. The hidden state is then fed into the network at the next time step via the recurrent connection (labelled with 'V'), e.g. from the hidden unit at time step $t-1$ to the hidden unit at time step $t$, and from the hidden unit at time step $t$ to the hidden unit at time step $t+1$. Note the weights and biases are shared across time steps. 

```{figure} https://upload.wikimedia.org/wikipedia/commons/b/b5/Recurrent_neural_network_unfold.svg
---
height: 300px
name: rnn-unfold
---
Unfolding a recurrent neural network across time steps
```

Let us see how RNNs work by going through an example adapted from the PyTorch tutorial [Classifying Names with a Character-Level RNN](https://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html) with some modifications.

## Get the data for _surname_ classification

A character-level RNN reads words as a series of characters and outputs a prediction and "hidden state" at each step, feeding its previous hidden state into each next step. We take the final prediction to be the output, i.e. which class the word belongs to. 

Specifically, we will train on a few thousand _surnames_ from 18 languages of origin, and predict which language a name is from based on the spelling.

Get ready by importing the APIs needed from respective libraries and setting the random seed for reproducibility.

In [ ]:
from __future__ import unicode_literals, print_function, division
from io import open
import glob
import os
import unicodedata
import string
import random
import time
import math

import torch
import torch.nn as nn
from torchvision.datasets.utils import download_and_extract_archive

import matplotlib.pyplot as plt
import matplotlib.ticker as ticker

torch.manual_seed(2022)
random.seed(2022)

Download and extract the data.

In [ ]:
root_dir = "./data/"
data_url = "https://download.pytorch.org/tutorial/data.zip"
download_and_extract_archive(data_url, root_dir)

Under the `data/names` directory, there are 18 text files named as "[Language].txt". Each file contains a bunch of names, one name per line, mostly romanized but we still need to convert from Unicode to ASCII.

Let us examine the extracted files.

In [ ]:
def findFiles(path):
    return glob.glob(path)


print(findFiles(root_dir + "data/names/*.txt"))

## Preprocess text data

### Build a dictionary for names

Turn a Unicode string to plain ASCII based on [a Stack Overflow post](https://stackoverflow.com/a/518232/2809427).

In [ ]:
all_letters = string.ascii_letters + " .,;'"
n_letters = len(all_letters)


def unicodeToAscii(s):
    return "".join(
        c
        for c in unicodedata.normalize("NFD", s)
        if unicodedata.category(c) != "Mn" and c in all_letters
    )


print(unicodeToAscii("Ślusàrski"))

Build the `category_lines dictionary`, which contains a list of names per language `{language: [names ...]}`, by reading the files and splitting into lines. 

In [ ]:
category_lines = {}
all_categories = []


# Read a file and split into lines
def readLines(filename):
    lines = open(filename, encoding="utf-8").read().strip().split("\n")
    return [unicodeToAscii(line) for line in lines]


for filename in findFiles(root_dir + "data/names/*.txt"):
    category = os.path.splitext(os.path.basename(filename))[0]
    all_categories.append(category)
    lines = readLines(filename)
    category_lines[category] = lines

n_categories = len(all_categories)

Let us examine the dictionary by printing a few names for Italian.

In [ ]:
print(category_lines["Italian"][:5])

### Code names into tensors using one-hot encoding

Next, we need to represent each letter in a name by a [one-hot](https://en.wikipedia.org/wiki/One-hot) vector. To represent a single letter, we use a "one-hot vector" of size ``<1 x n_letters>``. A one-hot vector is filled with 0s except for a 1 at index of the current letter, e.g. ``"b" = <0 1 0 0 0 ...>``.

To make a word we join a bunch of those into a 2D matrix ``<line_length x 1 x n_letters>``. That extra 1 dimension is because PyTorch assumes everything is in batches - we're just using a batch size of 1 here.

In [ ]:
# Find the letter index from all_letters, e.g. "a" = 0
def letterToIndex(letter):
    return all_letters.find(letter)


# Turn a letter into a <1 x n_letters> tensor
def letterToTensor(letter):
    tensor = torch.zeros(1, n_letters)
    tensor[0][letterToIndex(letter)] = 1
    return tensor


# Turn a line into a <line_length x 1 x n_letters> tensor
def lineToTensor(line):
    tensor = torch.zeros(len(line), 1, n_letters)
    for li, letter in enumerate(line):
        tensor[li][0][letterToIndex(letter)] = 1
    return tensor


print(letterToTensor("J"))
print(lineToTensor("Jones").size())

The following is how a single name is represented as a tensor.

In [ ]:
print(lineToTensor("Jones"))

## Define a recurrent neural network

We can implement a simple recurrent neural network (RNN) in PyTorch using regular feed-forward layers below, with two linear (fully-connected) layers operating on an input and hidden state and a LogSoftmax layer before the final output.

In [ ]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(RNN, self).__init__()

        self.hidden_size = hidden_size

        self.i2h = nn.Linear(input_size + hidden_size, hidden_size)  # input to hidden
        self.i2o = nn.Linear(input_size + hidden_size, output_size)  # input to output
        self.softmax = nn.LogSoftmax(dim=1)

    def forward(self, input, hidden):
        combined = torch.cat((input, hidden), 1)
        hidden = self.i2h(combined)
        output = self.i2o(combined)
        output = self.softmax(output)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, self.hidden_size)


n_hidden = 128
rnn = RNN(n_letters, n_hidden, n_categories)

This RNN is illustrated in {numref}`rnn-module` below.

```{figure} https://i.imgur.com/Z2xbySO.png
---
height: 300px
name: rnn-module
---
A recurrent neural network module with two linear (fully-connected) layers.
```

To run a step of this RNN, we need to provide an input (the tensor for the current letter in this example) and a previous hidden state (with initialisation to zeros). At each step, we get the output as the probability of each language and a next hidden state for the next
step.

In [ ]:
input = letterToTensor("A")
hidden = torch.zeros(1, n_hidden)

output, next_hidden = rnn(input, hidden)

For efficiency, we will not create a new tensor for every step. Instead, we will use ``lineToTensor`` rather than ``letterToTensor`` and then operate on slices of ``lineTensor``. 

In [ ]:
input = lineToTensor("Albert")
hidden = torch.zeros(1, n_hidden)

output, next_hidden = rnn(input[0], hidden)
print(output)

Here, the output is a ``<1 x 18>`` tensor (`n_categories`=18), where every item is the likelihood of that category (higher is more likely).

## Optimisation, training and testing

### Prepare for training

Before training, we make a few helper functions ready. The first is to interpret the output of the network, which we know to be a likelihood of each category. We can use ``Tensor.topk`` to get the index of the greatest value:

In [ ]:
def categoryFromOutput(output):
    top_n, top_i = output.topk(1)
    category_i = top_i[0].item()
    return all_categories[category_i], category_i


print(categoryFromOutput(output))

We also need a quick way to get a training example (a name and its language) randomly. We show 10 such random examples.

In [ ]:
def randomChoice(list):
    return list[random.randint(0, len(list) - 1)]


def randomTrainingExample():
    category = randomChoice(all_categories)
    line = randomChoice(category_lines[category])
    category_tensor = torch.tensor([all_categories.index(category)], dtype=torch.long)
    line_tensor = lineToTensor(line)
    return category, line, category_tensor, line_tensor


for i in range(10):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    print("category =", category, "/ line =", line)

### Choose a criterion and an optimiser

We choose the loss function ``nn.NLLLoss`` since the last layer of the RNN is ``nn.LogSoftmax``.

In [ ]:
learning_rate = 0.005  # A higher value may explode and a lower value may not learn.

criterion = nn.NLLLoss()
optimizer = torch.optim.SGD(rnn.parameters(), lr=learning_rate)

### Train the network

Now, we are ready to train this RNN by showing it a bunch of examples, having it make predictions, and computing the loss for gradient descent. 

Each loop of training will:

-  Create input and target tensors
-  Create a zeroed initial hidden state
-  Read each letter in and keep hidden state for next letter
-  Compare final output to target
-  Backpropagate
-  Return the output and loss

In [ ]:
def train(category_tensor, line_tensor):
    hidden = rnn.initHidden()

    optimizer.zero_grad()  # Clear the gradients

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    loss = criterion(output, category_tensor)
    loss.backward()
    optimizer.step()

    return output, loss.item()

Now, we run this RNN with a bunch of examples. Since the ``train`` function returns both the output and loss we can print its predictions and keep track of loss for plotting. Since there are 1000s of examples, we print only every ``print_every`` examples and take an average of the loss.

In [ ]:
n_iters = 60000
print_every = 2000
plot_every = 500

# Keep track of losses for plotting
current_loss = 0
all_losses = []


def timeSince(since):
    now = time.time()
    s = now - since
    m = math.floor(s / 60)
    s -= m * 60
    return "%dm %ds" % (m, s)


start = time.time()

for iter in range(1, n_iters + 1):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    output, loss = train(category_tensor, line_tensor)
    current_loss += loss

    # Print iter number, loss, name and prediction
    if iter % print_every == 0:
        predicted, predicted_i = categoryFromOutput(output)
        correct = "✓" if predicted == category else "✗ (%s)" % category
        print(
            "%d %d%% (%s) %.4f %s / %s %s"
            % (
                iter,
                iter / n_iters * 100,
                timeSince(start),
                loss,
                line,
                predicted,
                correct,
            )
        )

    # Add current loss avg to list of losses
    if iter % plot_every == 0:
        all_losses.append(current_loss / plot_every)
        current_loss = 0

### Plot the loss

Plotting the historical loss from ``all_losses`` shows how the network learned over time.

In [ ]:
plt.figure()
plt.plot(all_losses)

### Evaluate the results

To see how well the network performs on different categories, we will create a confusion matrix, indicating for every actual language (rows) which language the network has predicted to be (columns). To calculate the confusion matrix, a bunch of samples are run through the network with ``evaluate()``, which is the same as ``train()`` minus the backprop.

In [ ]:
# Keep track of correct predictions in a confusion matrix
confusion = torch.zeros(n_categories, n_categories)
n_confusion = 10000


# Just return an output given a line
def evaluate(line_tensor):
    hidden = rnn.initHidden()

    for i in range(line_tensor.size()[0]):
        output, hidden = rnn(line_tensor[i], hidden)

    return output


# Go through a bunch of examples and record which are correctly predicted
for i in range(n_confusion):
    category, line, category_tensor, line_tensor = randomTrainingExample()
    output = evaluate(line_tensor)
    predicted, predicted_i = categoryFromOutput(output)
    category_i = all_categories.index(category)
    confusion[category_i][predicted_i] += 1

# Normalize by dividing every row by its sum
for i in range(n_categories):
    confusion[i] = confusion[i] / confusion[i].sum()

# Set up plot
fig = plt.figure()
ax = fig.add_subplot(111)
cax = ax.matshow(confusion.numpy())
fig.colorbar(cax)

# Set up axes
ax.set_xticklabels([""] + all_categories, rotation=90)
ax.set_yticklabels([""] + all_categories)

# Force label at every tick
ax.xaxis.set_major_locator(ticker.MultipleLocator(1))
ax.yaxis.set_major_locator(ticker.MultipleLocator(1))

plt.show()

We can pick out bright spots off the main axis that show which languages it has most confusion with, e.g. Chinese for Korean. It seems to do very well with Greek, and very poorly with English (perhaps because of overlap with other languages). 

### Test on user input

We can also run the network on user input to see what it predicts.

In [ ]:
def predict(input_line, n_predictions=3):
    print("\n> %s" % input_line)
    with torch.no_grad():
        output = evaluate(lineToTensor(input_line))

        # Get top N categories
        topv, topi = output.topk(n_predictions, 1, True)
        predictions = []

        for i in range(n_predictions):
            value = topv[0][i].item()
            category_index = topi[0][i].item()
            print("(%.2f) %s" % (value, all_categories[category_index]))
            predictions.append([value, all_categories[category_index]])


predict("Hinton")
predict("Schmidhuber")
predict("Zhou")

Do these predictions make sense to you?

## Advanced RNNs

There are more advanced RNNs that can be used for sequence modelling, such as the [Long Short-Term Memory (LSTM)](https://en.wikipedia.org/wiki/Long_short-term_memory), [Gated Recurrent Unit (GRU)](https://en.wikipedia.org/wiki/Gated_recurrent_unit), and [Transformer](https://en.wikipedia.org/wiki/Transformer_(machine_learning_model)). These networks are more complex and are beyond the scope of this course. However, you can find more information about them in the [PyTorch documentation](https://pytorch.org/docs/stable/nn.html#recurrent-layers).


## Exercises

**1.** How is a recurrent neural network **different** from a convolutional neural network?

*Compare your answer with the solution below*

```{toggle}
CNNs are a type of neural networks that is particularly well suited to processing data that has a grid-like structure, such as images, whereas RNNs are particularly well suited to processing data that has a sequence structure, such as text.
```

**2.** What are the **two key ideas** behind RNNs?

*Compare your answer with the solution below*

```{toggle}
The two key ideas behind RNNs are recurrent connections and weight sharing.
```

**3.** What is the purpose of **recurrent connections** in RNNs?

*Compare your answer with the solution below*

```{toggle}
Recurrent connections allow information to flow through the network in a directed cycle, allowing the network to use its internal state to make predictions about the next element in the sequence.
```

**4.** What is the purpose of **weight sharing** in RNNs?

*Compare your answer with the solution below*

```{toggle}
Weight sharing in RNNs means that the same weights are used to process all time steps, which allows the network to efficiently represent patterns that are consistent across steps.
```